In [1]:
from src.data.mgf_tools.mgf_get import *
from src.config import *
from src.utils import *
import pandas as pd
from deepmol.splitters import MultiTaskStratifiedSplitter
from deepmol.compound_featurization import MorganFingerprint
from deepmol.datasets import SmilesDataset
from src.config import *

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
2025-08-04 20:09:38.488925: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-04 20:09:38.488975: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-04 20:09:38.490444: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-04 20:09:38.498827: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild Tens

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (/home/cgomes/miniforge3/envs/transformer/lib/python3.11/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
/home/cgomes/miniforge3/envs/transformer/lib/python3.11/site-packages/deepmol/compound_featurization/__init__.py:20: UserWarning: Mol2Vec not available. Please install it to use it. (pip install git+https://github.com/samoturk/mol2vec#egg=mol2vec)
  warnings.warn("Mol2Vec not available. Please install it to use it. "


# DATA SPLIT


In [2]:
mgf_data = mgf_path


In [3]:
spectra = mgf_get_spectra(mgf_data)

In [4]:
smiles_data = mgf_get_smiles(spectra)

smiles_df = pd.DataFrame(smiles_data)

In [5]:
processed_spectra = mgf_deconvoluter(mgf_data=spectra, mz_vocabs=mz_vocabs, min_num_peaks=5, max_num_peaks=max_num_peaks, noise_rmv_threshold=0.01, mass_error=0.01, allowed_spectral_entropy=True, log=False)


In [6]:
spectrum_ids = [spec[0] for spec in processed_spectra]
print(f"Espectros processados: {len(spectrum_ids)}")

filtered_smiles= smiles_df[smiles_df['spectrum_id'].isin(spectrum_ids)]
print(f"SMILES filtrados: {len(filtered_smiles)}")

Espectros processados: 276307
SMILES filtrados: 276307


In [7]:
smiles_list = filtered_smiles['smiles'].tolist()
ids_list = filtered_smiles['spectrum_id'].tolist()

In [8]:
dataset = SmilesDataset(smiles=smiles_list, ids=ids_list)
print(f"Dataset inicial: {len(dataset)} samples")

Dataset inicial: 276307 samples


In [9]:
dataset = MorganFingerprint().featurize(dataset)
print(f"Fingerprints gerados: {len(dataset)}")

MorganFingerprint: 100%|██████████| 276307/276307 [06:08<00:00, 750.68it/s]


Fingerprints gerados: 276307


In [10]:
dataset.X

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
dataset._y = dataset.X

In [12]:
dataset.y

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
train_dataset, val_dataset, test_dataset = MultiTaskStratifiedSplitter().train_valid_test_split(
    dataset, frac_train=0.8, frac_val=0.1, frac_test=0.1, seed=0)

In [14]:
print(f"SPLIT")
print(f"Train: {len(train_dataset)} samples")
print(f"Validation: {len(val_dataset)} samples")
print(f"Test: {len(test_dataset)} samples")

SPLIT
Train: 218543 samples
Validation: 29335 samples
Test: 28429 samples


In [15]:
def generate_stats(y_train: np.ndarray, y_test: np.ndarray, y_val: np.ndarray=None):
    """
    Parameters
    ----------
    y_train : np.ndarray
        Labels of the train set
    y_test : np.ndarray
        Labels of the test set
    y_val : np.ndarray, optional
        Labels of the validation set, by default None
    
    Returns
    -------
    Tuple[pd.DataFrame, Any]
        DataFrame with the stats of the split, styled table
    """
    y_test_sum = np.sum(y_test, axis=0)
    y_train_sum = np.sum(y_train, axis=0)

    sum_of_all = pd.DataFrame([y_train_sum, y_test_sum], index=["train", "test"])

    if y_val is not None:
        y_val_sum = np.sum(y_val, axis=0)
        sum_of_all = pd.DataFrame([y_train_sum, y_test_sum, y_val_sum], index=["train", "test", "validation"])
        sum_of_all.loc['Validation relative split', :] = sum_of_all.loc['validation', :] / (sum_of_all.loc['train', :] + sum_of_all.loc['test', :] + sum_of_all.loc['validation', :]) * 100
        sum_of_all.loc['Test relative split', :] = sum_of_all.loc['test', :] / (sum_of_all.loc['train', :] + sum_of_all.loc['test', :]+ sum_of_all.loc['validation', :]) * 100
        sum_of_all.loc['Train relative split', :] = sum_of_all.loc['train', :] / (sum_of_all.loc['train', :] + sum_of_all.loc['test', :]+ sum_of_all.loc['validation', :]) * 100

    else:
        sum_of_all.loc['Test relative split', :] = sum_of_all.loc['test', :] / (sum_of_all.loc['train', :] + sum_of_all.loc['test', :]) * 100
        sum_of_all.loc['Train relative split', :] = sum_of_all.loc['train', :] / (sum_of_all.loc['train', :] + sum_of_all.loc['test', :]) * 100

    df = pd.melt(sum_of_all.T.reset_index(), id_vars=['index']).rename(columns={'index': 'EC', 'value': 'Percentage of data'})
    if y_val is not None:
        df = df[(df["variable"]!="train") & (df["variable"]!="validation") & (df["variable"]!="test")]
    else: 
        df = df[(df["variable"]!="train") & (df["variable"]!="test")]

    df1 = sum_of_all.loc['Test relative split', :].describe()
    df2 = sum_of_all.loc['Train relative split', :].describe()
    if y_val is not None:
        df3 = sum_of_all.loc['Validation relative split', :].describe()
        stats_table = pd.concat([df1, df2, df3], axis=1)
    else:
        stats_table = pd.concat([df1, df2], axis=1)

    stats_table.drop(['count'], inplace=True)
    table_styled = stats_table.style.background_gradient(cmap="YlGn")
    

    return df, table_styled


train_labels = train_dataset.y
val_labels = val_dataset.y
test_labels = test_dataset.y


df, table_styled = generate_stats(train_labels, test_labels, val_labels)

table_styled

,Test relative split,Train relative split,Validation relative split
mean,10.751229,80.141132,9.107639
std,4.903469,7.514379,4.145522
min,1.450116,40.689655,1.065719
25%,7.545080,76.463060,6.347362
50%,9.969892,81.202314,8.532858
75%,12.844055,85.111579,10.952622
max,44.696970,96.470588,32.934435
